# Solving Linear Systems

## LU decomposition

LU decomposition is a matrix factorization technique that decomposes a square matrix $A$ into the product of a lower triangular matrix $L$ and an upper triangular matrix $U$, such that $A = L \cdot U$.

This decomposition is particularly useful for solving systems of linear equations $A \mathbf{x} = \mathbf{b}$, as it allows us to break down the problem into two simpler triangular systems:

1. Solve $L \mathbf{y} = \mathbf{b}$ for $\mathbf{y}$
2. Solve $U \mathbf{x} = \mathbf{y}$ for $\mathbf{x}$

In this notebook, we'll demonstrate how to perform LU decomposition and use it to solve linear systems using Python's NumPy library.

In [1]:
import numpy as np

In [2]:
def lu_decomposition(A):
    """
    Perform LU decomposition of matrix A.
    Returns L and U such that A = L @ U
    """
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy().astype(float)
    
    for i in range(n):
        for j in range(i+1, n):
            if U[i, i] == 0:
                raise ValueError("LU decomposition failed: zero pivot")
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j, :] -= factor * U[i, :]
    
    return L, U

def forward_substitution(L, b):
    """
    Solve L y = b for y
    """
    n = L.shape[0]
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i]
        for j in range(i):
            y[i] -= L[i, j] * y[j]
        y[i] /= L[i, i]
    return y

def backward_substitution(U, y):
    """
    Solve U x = y for x
    """
    n = U.shape[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for j in range(i+1, n):
            x[i] -= U[i, j] * x[j]
        x[i] /= U[i, i]
    return x

## What is LU Decomposition?

LU decomposition factors a matrix $A$ into:

$$A = L \cdot U$$

Where:
- $L$ is a lower triangular matrix with 1s on the diagonal
- $U$ is an upper triangular matrix

For a system $A \mathbf{x} = \mathbf{b}$, we can write:

$$L \cdot U \cdot \mathbf{x} = \mathbf{b}$$

Let $\mathbf{y} = U \cdot \mathbf{x}$, then:

$$L \cdot \mathbf{y} = \mathbf{b}$$

Solving for $\mathbf{y}$ is easy since $L$ is lower triangular, then solve $U \cdot \mathbf{x} = \mathbf{y}$ for $\mathbf{x}$.

In [3]:
# Example matrix A and vector b
A = np.array([[4, 3, 2],
              [1, 2, 1],
              [2, 1, 3]], dtype=float)

b = np.array([10, 5, 8], dtype=float)

print("Matrix A:")
print(A)
print("\nVector b:")
print(b)

Matrix A:
[[4. 3. 2.]
 [1. 2. 1.]
 [2. 1. 3.]]

Vector b:
[10.  5.  8.]


In [4]:
# Perform LU decomposition
L, U = lu_decomposition(A)

print("Lower triangular matrix L:")
print(L)
print("\nUpper triangular matrix U:")
print(U)
print("\nVerify A = L @ U:")
print(L @ U)

Lower triangular matrix L:
[[ 1.    0.    0.  ]
 [ 0.25  1.    0.  ]
 [ 0.5  -0.4   1.  ]]

Upper triangular matrix U:
[[4.   3.   2.  ]
 [0.   1.25 0.5 ]
 [0.   0.   2.2 ]]

Verify A = L @ U:
[[4. 3. 2.]
 [1. 2. 1.]
 [2. 1. 3.]]


## Solving the Linear System

To solve $A \mathbf{x} = \mathbf{b}$ using LU decomposition:

1. First, solve $L \mathbf{y} = \mathbf{b}$ for $\mathbf{y}$ (forward substitution)
2. Then, solve $U \mathbf{x} = \mathbf{y}$ for $\mathbf{x}$ (backward substitution)

Note: In some implementations, pivoting is used for numerical stability, but this assumes no pivoting is needed.

In [5]:
# Solve the system using LU decomposition
# First, solve L y = b
y = forward_substitution(L, b)
print("Intermediate vector y:")
print(y)

# Then, solve U x = y
x = backward_substitution(U, y)
print("\nSolution vector x:")
print(x)

# Verify the solution
print("\nVerification A @ x:")
print(A @ x)
print("Original b:")
print(b)

Intermediate vector y:
[10.   2.5  4. ]

Solution vector x:
[0.63636364 1.27272727 1.81818182]

Verification A @ x:
[10.  5.  8.]
Original b:
[10.  5.  8.]


## Cholesky Factorization

Cholesky factorization is a special case of LU decomposition that applies to symmetric positive definite matrices. It decomposes a matrix $A$ into the product of a lower triangular matrix $L$ and its transpose $L^T$, such that:

$$A = L \cdot L^T$$

Where:
- $L$ is a lower triangular matrix with positive diagonal elements
- $A$ must be symmetric ($A = A^T$) and positive definite (all eigenvalues positive)

This factorization is more efficient than general LU decomposition and is numerically stable. It's commonly used in applications like least squares problems, Kalman filtering, and Monte Carlo simulations.

For solving $A \mathbf{x} = \mathbf{b}$, we can write:

$$L \cdot L^T \cdot \mathbf{x} = \mathbf{b}$$

Let $\mathbf{y} = L^T \cdot \mathbf{x}$, then:

$$L \cdot \mathbf{y} = \mathbf{b}$$

Solving for $\mathbf{y}$ using forward substitution, then solve $L^T \cdot \mathbf{x} = \mathbf{y}$ using backward substitution (since $L^T$ is upper triangular).

In [6]:
def cholesky_decomposition(A):
    """
    Perform Cholesky decomposition of a symmetric positive definite matrix A.
    Returns L such that A = L @ L.T
    """
    n = A.shape[0]
    L = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i):
            L[i, j] = (A[i, j] - np.sum(L[i, :j] * L[j, :j])) / L[j, j]
        L[i, i] = np.sqrt(A[i, i] - np.sum(L[i, :i] ** 2))
    
    return L

# Example: Symmetric positive definite matrix
A_chol = np.array([[4, 2, 1],
                    [2, 5, 3],
                    [1, 3, 6]], dtype=float)

b_chol = np.array([7, 12, 15], dtype=float)

print("Symmetric positive definite matrix A:")
print(A_chol)
print("\nVector b:")
print(b_chol)

# Perform Cholesky decomposition
L = cholesky_decomposition(A_chol)

print("\nLower triangular matrix L:")
print(L)
print("\nVerify A = L @ L.T:")
print(L @ L.T)

Symmetric positive definite matrix A:
[[4. 2. 1.]
 [2. 5. 3.]
 [1. 3. 6.]]

Vector b:
[ 7. 12. 15.]

Lower triangular matrix L:
[[2.         0.         0.        ]
 [1.         2.         0.        ]
 [0.5        1.25       2.04633819]]

Verify A = L @ L.T:
[[4. 2. 1.]
 [2. 5. 3.]
 [1. 3. 6.]]


In [7]:
# Solve the system using Cholesky decomposition
# First, solve L y = b
y_chol = forward_substitution(L, b_chol)
print("Intermediate vector y:")
print(y_chol)

# Then, solve L.T x = y (backward substitution since L.T is upper triangular)
x_chol = backward_substitution(L.T, y_chol)
print("\nSolution vector x:")
print(x_chol)

# Verify the solution
print("\nVerification A @ x:")
print(A_chol @ x_chol)
print("Original b:")
print(b_chol)

Intermediate vector y:
[3.5        4.25       3.87887986]

Solution vector x:
[0.80597015 0.94029851 1.89552239]

Verification A @ x:
[ 7. 12. 15.]
Original b:
[ 7. 12. 15.]


## Iterative Methods for Solving Linear Systems

While direct methods like LU decomposition and Cholesky factorization provide exact solutions in a finite number of steps, iterative methods approximate the solution through successive refinements. These methods are particularly useful for large sparse matrices where direct methods become computationally expensive.

### Jacobi Method

The Jacobi method is the simplest iterative technique. It updates all components of the solution vector simultaneously using the values from the previous iteration:

$$x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j \neq i} a_{ij} x_j^{(k)} \right)$$

Convergence is guaranteed if the matrix is strictly diagonally dominant or symmetric positive definite.

### Gauss-Seidel Method

The Gauss-Seidel method improves upon Jacobi by using the most recently computed values within the same iteration:

$$x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij} x_j^{(k)} \right)$$

This method generally converges faster than Jacobi for the same matrices.

### Successive Over-Relaxation (SOR) Method

SOR introduces a relaxation factor ω to accelerate convergence:

$$x_i^{(k+1)} = (1 - \omega) x_i^{(k)} + \frac{\omega}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij} x_j^{(k)} \right)$$

- For ω = 1, SOR reduces to Gauss-Seidel
- For 0 < ω < 1, it's under-relaxation (slower convergence)
- For 1 < ω < 2, it's over-relaxation (potentially faster convergence)

The optimal ω depends on the matrix properties and is typically found experimentally.

In [8]:
def jacobi_method(A, b, x0=None, tol=1e-10, max_iter=100):
    """
    Solve A x = b using Jacobi iterative method.
    Returns solution x and number of iterations.
    """
    n = A.shape[0]
    if x0 is None:
        x0 = np.zeros(n)
    x = x0.copy()
    
    for k in range(max_iter):
        x_new = np.zeros(n)
        for i in range(n):
            x_new[i] = (b[i] - np.sum(A[i, :] * x) + A[i, i] * x[i]) / A[i, i]
        
        if np.linalg.norm(x_new - x) < tol:
            return x_new, k + 1
        x = x_new
    
    return x, max_iter

def gauss_seidel_method(A, b, x0=None, tol=1e-10, max_iter=100):
    """
    Solve A x = b using Gauss-Seidel iterative method.
    Returns solution x and number of iterations.
    """
    n = A.shape[0]
    if x0 is None:
        x0 = np.zeros(n)
    x = x0.copy()
    
    for k in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            x[i] = (b[i] - np.sum(A[i, :i] * x[:i]) - np.sum(A[i, i+1:] * x_old[i+1:])) / A[i, i]
        
        if np.linalg.norm(x - x_old) < tol:
            return x, k + 1
    
    return x, max_iter

def sor_method(A, b, omega=1.25, x0=None, tol=1e-10, max_iter=100):
    """
    Solve A x = b using Successive Over-Relaxation (SOR) method.
    Returns solution x and number of iterations.
    """
    n = A.shape[0]
    if x0 is None:
        x0 = np.zeros(n)
    x = x0.copy()
    
    for k in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            sigma = np.sum(A[i, :i] * x[:i]) + np.sum(A[i, i+1:] * x_old[i+1:])
            x[i] = (1 - omega) * x_old[i] + (omega / A[i, i]) * (b[i] - sigma)
        
        if np.linalg.norm(x - x_old) < tol:
            return x, k + 1
    
    return x, max_iter

In [9]:
# Example: Diagonally dominant matrix for iterative methods
A_iter = np.array([[10, 2, 1],
                   [1, 8, 2],
                   [2, 1, 9]], dtype=float)

b_iter = np.array([13, 11, 12], dtype=float)

print("Matrix A (diagonally dominant):")
print(A_iter)
print("\nVector b:")
print(b_iter)

# Initial guess
x0 = np.zeros(3)

# Jacobi method
x_jacobi, iter_jacobi = jacobi_method(A_iter, b_iter, x0)
print(f"\nJacobi method converged in {iter_jacobi} iterations")
print("Solution:", x_jacobi)
print("Verification A @ x:", A_iter @ x_jacobi)

# Gauss-Seidel method
x_gs, iter_gs = gauss_seidel_method(A_iter, b_iter, x0)
print(f"\nGauss-Seidel method converged in {iter_gs} iterations")
print("Solution:", x_gs)
print("Verification A @ x:", A_iter @ x_gs)

# SOR method with omega = 1.25
x_sor, iter_sor = sor_method(A_iter, b_iter, omega=1.25, x0=x0)
print(f"\nSOR method (ω=1.25) converged in {iter_sor} iterations")
print("Solution:", x_sor)
print("Verification A @ x:", A_iter @ x_sor)

Matrix A (diagonally dominant):
[[10.  2.  1.]
 [ 1.  8.  2.]
 [ 2.  1.  9.]]

Vector b:
[13. 11. 12.]

Jacobi method converged in 23 iterations
Solution: [1. 1. 1.]
Verification A @ x: [13. 11. 12.]

Gauss-Seidel method converged in 12 iterations
Solution: [1. 1. 1.]
Verification A @ x: [13. 11. 12.]

SOR method (ω=1.25) converged in 23 iterations
Solution: [1. 1. 1.]
Verification A @ x: [13. 11. 12.]


## Conclusion

This notebook has covered both direct and iterative methods for solving linear systems:

**Direct Methods:**
- LU decomposition: Works for general square matrices, decomposes in $O(n^3)$ time
- Cholesky factorization: Specialized for symmetric positive definite matrices, more efficient than LU

**Iterative Methods:**
- Jacobi method: Simple simultaneous updates, converges for diagonally dominant matrices
- Gauss-Seidel method: Uses latest values within iteration, generally faster than Jacobi
- SOR method: Relaxation-based acceleration, optimal ω can significantly improve convergence

Direct methods provide exact solutions in finite steps but require $O(n^3)$ operations. Iterative methods approximate solutions through successive refinements and are preferred for large sparse systems where direct methods become impractical.

The choice of method depends on matrix properties (sparsity, structure, conditioning) and problem size. Understanding these fundamental algorithms helps in selecting appropriate numerical methods and appreciating the optimizations in libraries like NumPy and SciPy.

Experiment with different matrices, relaxation factors, and convergence tolerances to see how these methods perform!

## Glossary

- **LU Decomposition**: A matrix factorization technique that decomposes a square matrix $A$ into the product of a lower triangular matrix $L$ and an upper triangular matrix $U$ ($A = L \cdot U$).

- **Lower Triangular Matrix**: A square matrix where all entries above the main diagonal are zero.

- **Upper Triangular Matrix**: A square matrix where all entries below the main diagonal are zero.

- **Cholesky Factorization**: A special LU decomposition for symmetric positive definite matrices, where $U = L^T$ ($A = L \cdot L^T$).

- **Symmetric Matrix**: A square matrix that equals its transpose ($A = A^T$).

- **Positive Definite Matrix**: A symmetric matrix where all eigenvalues are positive, ensuring $x^T A x > 0$ for all non-zero vectors $x$.

- **Diagonal Dominance**: A property where the absolute value of each diagonal element is greater than or equal to the sum of absolute values of other elements in its row/column, guaranteeing convergence of iterative methods.

- **Jacobi Method**: An iterative technique that updates all components of the solution vector simultaneously using the values from the previous iteration: $x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j \neq i} a_{ij} x_j^{(k)} \right)$.

- **Gauss-Seidel Method**: An iterative method that uses the most recently computed values within the same iteration: $x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij} x_j^{(k)} \right)$.

- **Successive Over-Relaxation (SOR)**: An accelerated iterative method that combines Gauss-Seidel with a relaxation factor $\omega$: $x_i^{(k+1)} = (1 - \omega) x_i^{(k)} + \frac{\omega}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij} x_j^{(k)} \right)$.

- **Relaxation Factor ($\omega$)**: A parameter in SOR method; $\omega=1$ gives Gauss-Seidel, $0 < \omega < 1$ under-relaxes, $1 < \omega < 2$ over-relaxes for potentially faster convergence.

- **Convergence**: The process by which an iterative method approaches the exact solution within a specified tolerance.

- **Tolerance ($\epsilon$)**: The acceptable error threshold for stopping an iterative method (e.g., when $\|x_{\text{new}} - x_{\text{old}}\| < \epsilon$).

- **Iteration**: A single step in an iterative algorithm that refines the solution approximation.

- **Forward Substitution**: A method to solve lower triangular systems $L y = b$ by solving for $y$ sequentially from top to bottom.

- **Backward Substitution**: A method to solve upper triangular systems $U x = y$ by solving for $x$ sequentially from bottom to top.

- **Pivoting**: Row/column exchanges in LU decomposition to improve numerical stability by avoiding division by small numbers.